In [ ]:
from chromatic import *
import emcee
import corner
import speclite as speclite
import glob as glob
from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from PyAstronomy import pyasl
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('tableau-colorblind10')

In [ ]:
def model_spectrum(parameters,modeltype='PHOENIX',wavelength=None,**kwargs):
    
    f_spot, delta_f_spot, T_spot, T_amb = parameters 

    S_spot = get_phoenix_photons(temperature=int(T_spot),wavelength=wavelength,
                                    logg=4.4,metallicity=0.0)
    S_amb = get_phoenix_photons(temperature=int(T_amb),wavelength=wavelength,
                                   logg=4.4,metallicity=0.0)
    
    model = f_spot*S_spot[1] + (1.-f_spot)*S_amb[1]
    
    return S_spot[0],model
photons/(s * m**2 * nm)

In [ ]:
# wavelength = np.linspace(3500,9000,1000)*u.angstrom
w,phot_spectrum = get_phoenix_photons(temperature=int(3900),R=100,
                                    logg=4.4,metallicity=0.0)
w,spot_spectrum = get_phoenix_photons(temperature=int(3100),R=100,
                                    logg=4.4,metallicity=0.0)

max_combined_spec = (0.35)*spot_spectrum + (0.65)*phot_spectrum
min_combined_spec = (0.45)*spot_spectrum + (0.55)*phot_spectrum

In [ ]:
contrast = 1.-(spot_spectrum/phot_spectrum)
ds_over_s = -0.05 * ( contrast / ( 1.- 0.40*contrast ) )
semi_amplitude = np.abs(ds_over_s)

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(14,10))
fig.suptitle('Spotted Spectra of AU Mic',fontsize=25)
axs[0,0].plot(w*1e3,phot_spectrum,label='3900 K Photosphere Spectrum',color='blue')
axs[0,0].plot(w*1e3,spot_spectrum,label='3100 K Spot Spectrum',color='red')
axs[0,0].plot(w*1e3,max_combined_spec,label=r'Avg Spec with $f_{\rm{spot}}=0.35$')
axs[0,0].plot(w*1e3,min_combined_spec,label=r'Avg Spec with $f_{\rm{spot}}=0.45$')
axs[0,0].axvspan(400,540,alpha=0.2,color='orange')
axs[0,0].axvspan(560,670,alpha=0.2,color='teal')
axs[0,0].axvspan(690,820,alpha=0.2,color='purple')
axs[0,0].axvspan(551,888,alpha=0.2,color='gray')
axs[0,0].set_ylabel(r'Flux photons/(s * m^2 * nm)')
axs[0,0].set_xlim(200,3000)
axs[0,0].legend(loc='upper right')

axs[0,1].plot(w*1e3,phot_spectrum,color='blue')
axs[0,1].plot(w*1e3,spot_spectrum,color='red')
axs[0,1].plot(w*1e3,max_combined_spec)
axs[0,1].plot(w*1e3,min_combined_spec)
axs[0,1].axvspan(400,540,label='SDSS g\' Bandpass',alpha=0.2,color='orange')
axs[0,1].axvspan(560,670,label='SDSS r\' Bandpass',alpha=0.2,color='teal')
axs[0,1].axvspan(690,820,label='SDSS i\' Bandpass',alpha=0.2,color='purple')
axs[0,1].axvspan(551,888,label='NRES Bandpass',alpha=0.2,color='gray')
axs[0,1].set_xlim(350,900)
axs[0,1].legend(loc='upper left')

axs[1,0].plot(w*1e3,max_combined_spec/min_combined_spec,label='Peak/Trough Flux',color='black')
axs[1,0].axvspan(400,540,alpha=0.2,color='orange')
axs[1,0].axvspan(560,670,alpha=0.2,color='teal')
axs[1,0].axvspan(690,820,alpha=0.2,color='purple')
axs[1,0].axvspan(551,888,alpha=0.2,color='gray')
axs[1,0].set_xlabel(r'Wavelength (nm)')
axs[1,0].set_ylabel(r'Relative Flux (phase max / phase min)')
axs[1,0].set_xlim(200,3000)
axs[1,0].legend(loc='upper right')

axs[1,1].plot(w*1e3,semi_amplitude*100,label='Semi-amplitude of rotational variability',color='black')
axs[1,1].axvspan(400,540,alpha=0.2,color='orange')
axs[1,1].axvspan(560,670,alpha=0.2,color='teal')
axs[1,1].axvspan(690,820,alpha=0.2,color='purple')
axs[1,1].axvspan(551,888,alpha=0.2,color='gray')
axs[1,1].set_xlabel(r'Wavelength (nm)')
axs[1,1].set_ylabel(r'Variability (%)')
axs[1,1].set_xlim(350,900)
axs[1,1].legend(loc='upper left')

plt.savefig('../figs/AU_Mic_spectral_variability.png',dpi=300)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))
fig.suptitle('Example Photosphere and Spot Spectra',fontsize=20)
axs[0].plot(w*1e3,phot_spectrum/1e22,label='3900 K Photosphere Spectrum',color='blue')
axs[0].plot(w*1e3,spot_spectrum/1e22,label='3100 K Spot Spectrum',color='red')
axs[0].plot(w*1e3,max_combined_spec/1e22,label=r'Spectrum with $f_{\rm{spot}}=0.35$',color='green')
axs[0].axvspan(400,540,alpha=0.2,color='orange')
axs[0].axvspan(560,670,alpha=0.2,color='teal')
axs[0].axvspan(690,820,alpha=0.2,color='purple')
axs[0].axvspan(551,888,alpha=0.2,color='gray')
axs[0].set_ylabel(r'Flux [x10$^{22}$ photons/(s * m$^2$ * nm)]')
axs[0].set_xlim(200,3000)
axs[0].legend(loc='upper right',fontsize=12)

axs[1].plot(w*1e3,phot_spectrum/1e22,color='blue')
axs[1].plot(w*1e3,spot_spectrum/1e22,color='red')
axs[1].plot(w*1e3,max_combined_spec/1e22,color='green')
axs[1].axvspan(400,540,label='SDSS g\' Bandpass',alpha=0.2,color='orange')
axs[1].axvspan(560,670,label='SDSS r\' Bandpass',alpha=0.2,color='teal')
axs[1].axvspan(690,820,label='SDSS i\' Bandpass',alpha=0.2,color='purple')
axs[1].axvspan(551,888,label='NRES Bandpass',alpha=0.2,color='gray')
axs[1].set_xlim(350,900)
axs[1].legend(loc='upper left',fontsize=12)

plt.savefig('../figs/AU_Mic_spectral_components.png',dpi=100)

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(14,10))
fig.suptitle('Spotted Spectra of AU Mic',fontsize=25)
axs[0,0].plot(w*1e3,phot_spectrum,label='3900 K Photosphere Spectrum',color='blue')
axs[0,0].plot(w*1e3,spot_spectrum,label='3100 K Spot Spectrum',color='red')
axs[0,0].plot(w*1e3,max_combined_spec,label=r'Avg Spec with $f_{\rm{spot}}=0.35$')
axs[0,0].plot(w*1e3,min_combined_spec,label=r'Avg Spec with $f_{\rm{spot}}=0.45$')
axs[0,0].axvspan(400,540,alpha=0.2,color='orange')
axs[0,0].axvspan(560,670,alpha=0.2,color='teal')
axs[0,0].axvspan(690,820,alpha=0.2,color='purple')
axs[0,0].axvspan(551,888,alpha=0.2,color='gray')
axs[0,0].set_ylabel(r'Flux photons/(s * m^2 * nm)')
axs[0,0].set_xlim(200,3000)
axs[0,0].legend(loc='upper right')

axs[0,1].plot(w*1e3,phot_spectrum,color='blue')
axs[0,1].plot(w*1e3,spot_spectrum,color='red')
axs[0,1].plot(w*1e3,max_combined_spec)
axs[0,1].plot(w*1e3,min_combined_spec)
axs[0,1].axvspan(400,540,label='SDSS g\' Bandpass',alpha=0.2,color='orange')
axs[0,1].axvspan(560,670,label='SDSS r\' Bandpass',alpha=0.2,color='teal')
axs[0,1].axvspan(690,820,label='SDSS i\' Bandpass',alpha=0.2,color='purple')
axs[0,1].axvspan(551,888,label='NRES Bandpass',alpha=0.2,color='gray')
axs[0,1].set_xlim(350,900)
axs[0,1].legend(loc='upper left')

axs[1,0].plot(w*1e3,max_combined_spec/min_combined_spec,label='Peak/Trough Flux',color='black')
axs[1,0].axvspan(400,540,alpha=0.2,color='orange')
axs[1,0].axvspan(560,670,alpha=0.2,color='teal')
axs[1,0].axvspan(690,820,alpha=0.2,color='purple')
axs[1,0].axvspan(551,888,alpha=0.2,color='gray')
axs[1,0].set_xlabel(r'Wavelength (nm)')
axs[1,0].set_ylabel(r'Relative Flux (phase max / phase min)')
axs[1,0].set_xlim(200,3000)
axs[1,0].legend(loc='upper right')

axs[1,1].plot(w*1e3,semi_amplitude*100,label='Semi-amplitude of rotational variability',color='black')
axs[1,1].axvspan(400,540,alpha=0.2,color='orange')
axs[1,1].axvspan(560,670,alpha=0.2,color='teal')
axs[1,1].axvspan(690,820,alpha=0.2,color='purple')
axs[1,1].axvspan(551,888,alpha=0.2,color='gray')
axs[1,1].set_xlabel(r'Wavelength (nm)')
axs[1,1].set_ylabel(r'Variability (%)')
axs[1,1].set_xlim(350,900)
axs[1,1].legend(loc='upper left')

plt.savefig('../figs/AU_Mic_spectral_variability.png',dpi=300)